In [1]:
import sys
import os
path = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(path)
sys.path.append(path + '/api')

import geopandas as gpd
from api.app.jhm_metric_calcs.jhm_metric import main
import networkx as nx
from api.app.constants import CRS_WGS84

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
path

'/home/gk/vscode/rita_project_4/industrial-location'

In [3]:
# read dataset with avg rent price and house locations (geometry)
city_name = 'saint-petersburg'
g_type='drive'

# read drive nx graph
G_d = nx.read_graphml(f'/home/gk/vscode/rita_project_4/industrial-location/api/app/provisions_data/saint-petersburg_prov/G_drive.graphml')

house_prices = gpd.read_parquet(f"/home/gk/vscode/rita_project_4/industrial-location/api/app/provisions_data/{city_name}_prov/houses_price_demo_prov.parquet")
house_prices = house_prices.to_crs(G_d.graph["crs"])



In [4]:
G_d.graph['crs']

'epsg:32636'

In [5]:
list(G_d.nodes(data=True))[3]

('1', {'x': 351066.09, 'y': 6643191.31})

In [6]:
# define company's potential location
# and convert coordinates crs to local crs

company_location = {"lat": 59.869047, "lon": 30.206686}

In [7]:
class Worker():
    speciality: str
    salary: int

worker1 = Worker()
worker1.speciality = "worker_1"
worker1.salary = 10_000

worker2 = Worker()
worker2.speciality = "worker_2"
worker2.salary = 80_000

In [8]:
worker_and_salary = [worker1, worker2]

In [10]:
res = main(
    gdf_houses=house_prices,
    worker_and_salary=worker_and_salary,
    company_location=company_location,
    graph=G_d,
    cell_size_meters=500)



 K: [0, 1.032] 

 D: 0.516 
 defaultdict(<class 'float'>, {'worker_1_avg': 0, 'worker_2_avg': 1.032})


In [20]:
import json
json.loads(res["gdfs"]["worker_2"]["grid"])

{'type': 'FeatureCollection',
 'features': [{'id': '1206',
   'type': 'Feature',
   'properties': {'Iq_mean': 0.0, 'P_avg_mean': 0.2, 'Idx_mean': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[29.620292694149374, 60.201409102935415],
      [29.620754016551917, 60.19692741828263],
      [29.611751969538112, 60.19669748107517],
      [29.611289423937816, 60.2011791242286],
      [29.620292694149374, 60.201409102935415]]]}},
  {'id': '1346',
   'type': 'Feature',
   'properties': {'Iq_mean': 0.0, 'P_avg_mean': 0.0, 'Idx_mean': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[29.62699342214091, 60.22404702367286],
      [29.627454252506173, 60.21956532349047],
      [29.618445938878075, 60.21933578972044],
      [29.61798388331718, 60.22381744845674],
      [29.62699342214091, 60.22404702367286]]]}},
  {'id': '1480',
   'type': 'Feature',
   'properties': {'Iq_mean': 0.0, 'P_avg_mean': 0.0, 'Idx_mean': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates'

In [27]:
import json

gpd.GeoDataFrame.from_features(
    json.loads(res["gdfs"]["worker_2"]["house_points"]), crs=CRS_WGS84
)

,geometry,avg_m2_price_rent,price,P_clinic,P_hospitals,P_kindergarten,P_mfc,P_schools,accs_time,log_accs_time,Iq,P_avg,Idx
0,POINT (30.31326 59.92869),1190.476190,25000,1,1,1.0,0.0,1.0,41.7,1.620,0.495,0.8,0.6475
1,POINT (30.19203 59.84717),538.461538,35000,1,1,1.0,0.0,1.0,19.5,1.290,0.435,0.8,0.6175
2,POINT (30.31716 59.92467),2000.000000,32000,1,1,0.0,1.0,0.0,41.0,1.613,0.355,0.6,0.4775
3,POINT (30.35469 59.92743),2133.333333,32000,1,1,1.0,1.0,1.0,46.9,1.671,0.332,1.0,0.6660
4,POINT (30.12793 59.84049),500.000000,18000,0,1,0.0,1.0,0.0,34.2,1.534,0.655,0.4,0.5275
...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,POINT (30.39363 59.84422),600.000000,27000,1,1,1.0,0.0,1.0,45.6,1.659,0.439,0.8,0.6195
233,POINT (30.39560 59.85169),566.666667,17000,1,1,1.0,0.0,1.0,46.1,1.664,0.647,0.8,0.7235
234,POINT (30.40713 59.85843),658.536585,27000,0,1,1.0,0.0,0.0,45.6,1.659,0.439,0.4,0.4195
235,POINT (30.41301 59.82754),667.792793,25333,1,1,0.0,1.0,1.0,52.7,1.722,0.454,0.8,0.6270


In [19]:
gdf2

,geometry,Iq_mean,P_avg_mean,Idx_mean
0,"POLYGON ((29.62029 60.20141, 29.62075 60.19693...",0.0,0.200,0
1,"POLYGON ((29.62699 60.22405, 29.62745 60.21957...",0.0,0.000,0
2,"POLYGON ((29.63646 60.21979, 29.63692 60.21531...",0.0,0.000,0
3,"POLYGON ((29.64822 60.19313, 29.64868 60.18865...",0.0,0.000,0
4,"POLYGON ((29.66531 60.20255, 29.66577 60.19807...",0.0,0.000,0
...,...,...,...,...
1436,"POLYGON ((30.64265 59.78785, 30.64296 59.78337...",0.0,0.400,0
1437,"POLYGON ((30.65186 59.78353, 30.65218 59.77904...",0.0,0.520,0
1438,"POLYGON ((30.66107 59.77920, 30.66139 59.77471...",0.0,0.360,0
1439,"POLYGON ((30.66076 59.78369, 30.66107 59.77920...",0.0,0.400,0


In [ ]:
gdf2['Idx'] = gdf2['Idx'].round(2)

In [ ]:
gdf2.to_file('gdf22.geojson')